# Covid and its financial impacts

In this notebook, you will compile the visualizations from the analysis into functions that can be used for a Panel dashboard.

In [87]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [88]:
file_path = Path("Data\owid-covid-data-20210928.csv")
covid_data = pd.read_csv(file_path)
#creating a new field 'Date' field off the back of 'date' and making it only as a date format of YYYY-MM-DD
covid_data['Date'] = pd.to_datetime(covid_data['date']).dt.date 

In [89]:
#covid_data
analysis = covid_data[['date','continent','location','total_cases','population_density','gdp_per_capita','reproduction_rate','total_vaccinations_per_hundred']]#.set_index('Date')
# removing any noise from the data set
analysis = analysis[(analysis['location'] != analysis['continent']) & (~analysis['continent'].isnull())]
analysis = analysis.fillna(0)
#analysis.info()
continent = analysis.groupby(['location'])['total_cases','population_density','gdp_per_capita','reproduction_rate','total_vaccinations_per_hundred'].mean().reset_index().sort_values(by='total_cases')
continent.head()

,location,total_cases,population_density,gdp_per_capita,reproduction_rate,total_vaccinations_per_hundred
134,Montserrat,0.0,0.000,0.0,0.0,4.142342
201,Tokelau,0.0,0.000,0.0,0.0,2.463509
149,Northern Cyprus,0.0,0.000,0.0,0.0,2.673131
28,British Virgin Islands,0.0,207.973,0.0,0.0,11.335943
82,Guernsey,0.0,0.000,0.0,0.0,37.056885


In [119]:
def gdp_per_capita():
    
    output = px.parallel_coordinates(continent, color='gdp_per_capita', title ='Multi-Dimensional plot vs GDP per Capita',)
    
    return output
def vaccinations_gdp_per_capita():
    
    fig3 = px.scatter(
    continent,
    y='gdp_per_capita',
    x='total_vaccinations_per_hundred',
    color = 'total_vaccinations_per_hundred',
    trendline="ols",
    title ='GDP per Capita vs Total Vaccinations per Hundred',
        hover_data=['location'],
        marginal_y="histogram"
)

    return fig3

def pop_density_total_cases():
    fig = px.scatter(continent, x="population_density", y="total_cases", trendline="ols", color="total_cases", hover_data=['location'], title ='Population Density vs Total Cases', log_x=True, 
                     #log_y=True
                    )
    return fig

In [120]:
parallel_coordinates = pn.Row(
    '## Tab 1',
    gdp_per_capita()
)

tab2 = pn.Row(  
    '## Tab2',
    vaccinations_gdp_per_capita(),
    pop_density_total_cases()
)

In [123]:
tabs = pn.Tabs(
    #("Welcome", welcome_column),
    ("JC: Multi-dimensional coordinates",parallel_coordinates),
    #("Parallel Categories and Coordinates", fig3),
    #("Average Cost in San Francisco", average_cost_sf),
    ("JC: Further analysis of Covid data", tab2),
    #("Sunburst", sunburst_column)
)


# Create the dashboard
tabs

Tabs
    [0] Row
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})
    [1] Row
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True}, viewport={'xaxis.range': [-6.270153...}, visible=False)
        [2] Plotly(Figure, relayout_data={'autosize': True}, viewport={'xaxis.range': [-1.198698...}, visible=False)